### cone_density: a tool for predicting cone density and size

A simple tool for interpolating cone photoreceptor density in the human retina. Using data from Curcio et al.'s 1990 paper *Human photoreceptor topography*.

To use, download and unzip the directory somewhere in your python path. Then, to estimate the cone density and row spacing at a given eccentricity, instantiate a `ConeDensityInterpolator` object and call its `get_density_and_rowspacing` function on the target x- and y-coordinates, specified in degrees of visual angle. The function returns a 2-tuple, `(density, row_spacing)`, where density is cone density in **mm<sup>-2</sup>** and row_spacing is in **m**.

#### An example:

In [1]:
from cone_density import ConeDensityInterpolator
cdi = ConeDensityInterpolator()
nasal_5_deg = -5
inferior_3_deg = 3.0
nasal_5_deg = -5.0
density,row_spacing = cdi.get_density_and_rowspacing(nasal_5_deg,inferior_3_deg)
print density # outputs 13302.348128 (cones/mm^2)
print row_spacing # outputs 8.06861314705e-06 (m)
print row_spacing*1e6 # outputs 8.0686131470546094 (um)

13302.348128009222
8.06865527709446e-06
8.06865527709446


#### Some conventions:

1. Nasal and superior eccentricities are specified with negative values, while temporal and inferior eccentricities are specified with positive ones.

2. The original paper gives densities in (**mm<sup>-2</sup>**) as a function of **mm**. The `get_density_and_rowspacing` function takes eccentricity in degrees and converts to **mm** using a factor of **300 &mu;m/deg**.

[Curcio, Christine A., et al. "Human photoreceptor topography." *Journal of Comparative Neurology* 292.4 (1990): 497-523.](https://www.ncbi.nlm.nih.gov/pubmed/2324310)

#### Maps of cone density and row spacing generated by `ConeDensityInterpolator.test()`:

![Cone density as a function of eccentricity in a central 20 x 20 deg square](./maps/density.png)

![Cone row spacing as a function of eccentricity in a central 20 x 20 deg square](./maps/row_spacing.png)

#### Interconversion of density, diameter, and row spacing

The function interpolates cone densities from Curcio et al's data, but returns both density and row spacing. Here's how the latter is computed from the former. Let's assume the cones are packed in a perfect hexagonal (or triangular) lattice. If they were hexagonal in shape, the number of cones would be the ratio of the total area to the area of one hexagon. Since the cones are circular, we need to divide the total area by the area of the smallest hexagon in which the cone could be inscribed. If $r$ is the cone's radius, the area of this hexagon can be expressed in terms of $r$, since the circle's diameter is equal to the length of a line segment normal to two opposite sides of the hexagon:

$$A_h = \frac{3\sqrt{3}}{2}\left[\frac{r}{\cos(\pi/6)}\right]^2\mathrm{.}$$

If the area of the inscribed circle is $A_c = \pi r^2$, then the ratio $A_c/A_h$ is the unitless [packing density](https://en.wikipedia.org/wiki/Packing_problems), $\phi = 0.907$.

Now, given a cone density $D$ given in $mm^{-2}$, we can calculate the radius $r$ of the circle which would be required to pack it perfectly:

$$r = \sqrt{\frac{\phi}{\pi D}}\mathrm{.}$$

Lastly, when we are analyzing cone images, we are often interested in the dominant spatial frequency, which is determined not by the center-to-center spacing, or diameter of the cones, but by the spacing between their rows, $S$, which is given by:

$$S = (2*r)*\cos(\pi/6)\mathrm{.}$$.

Let's check that the program works correctly:

In [2]:
from cone_density import ConeDensityInterpolator
import numpy as np
cdi = ConeDensityInterpolator()
nasal_5_deg = -5
inferior_3_deg = 3.0
nasal_5_deg = -5.0
D,S = cdi.get_density_and_rowspacing(nasal_5_deg,inferior_3_deg)
print D # outputs 13302.348128 (cones/mm^2)
phi = 0.907
expected_r = np.sqrt(phi/(D*np.pi))
expected_S = 2*expected_r*np.cos(np.pi/6.0)
# now print them, remembering that the function get_density_and_rowspacing returns row spacing with units m
print S,expected_S*1e-3 # slightly different due to rounding of phi above; true value is calculated in cone_density

13302.348128009222
8.06865527709446e-06 8.06910152713089e-06
